In [ ]:
!pip3 install transformers

# **Import libararies**

In [1]:
import nltk
from sklearn.preprocessing import LabelEncoder

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
"""Import the bert model from transformers"""
import transformers
# from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer,AdamW, get_linear_schedule_with_warmup
import torch

"""Import necessary libraries"""
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection  import train_test_split
from torch import nn, optim
from torch.utils.data import Dataset,DataLoader
import torch.nn.functional as F
from pylab import rcParams
from matplotlib import rc
import spacy
import nltk
import re
# nltk.download('stopwords')
from sklearn.preprocessing import LabelEncoder


# nltk.download('wordnet')
# nltk.download('omw-1.4')

# **Initialize and load data**

In [3]:
"""Intialize the  constants"""
RANDOM_SEED=42
class_names=42 #since there are 41 creators in total
BATCH_SIZE=16
MAX_LEN=384
EPOCHS=5
LEARNING_RATE=2e-5
PRE_TRAINED_MODEL_NAME='distilbert-base-uncased'
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [4]:
data=pd.read_csv("superheroes_nlp_dataset.csv",encoding="utf-8")
dataset=data[['history_text','creator']]

# **Load the pre-trained model**

In [5]:
"""load the pretrained model from transformers and tweak it with our extracted features """
class DistillBERTClass(torch.nn.Module):

   def __init__(self,n_classes,PRE_TRAINED_MODEL_NAME):
     super(DistillBERTClass,self).__init__()
     self.bert=transformers.DistilBertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
     self.drop=nn.Dropout(p=0.2)
     self.drop = torch.nn.Dropout(0.3)
     self.out = torch.nn.Linear(self.bert.config.hidden_size,n_classes)
    
   def forward(self,input_ids, attention_mask):
        distilbert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = distilbert_output[0]  # (bs, seq_len, dim)
        pooled_output = hidden_state[:, 0]  # (bs, dim)
        output_1 = self.drop(pooled_output)
        output = self.out(output_1)
        return output

In [6]:
"""load the bert model, tokenizer from transformers"""
bert_model=DistilBertForSequenceClassification.from_pretrained(PRE_TRAINED_MODEL_NAME)
model=DistillBERTClass(class_names,PRE_TRAINED_MODEL_NAME)
model.to(device)
tokenizer=DistilBertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.w

# **Preprocess the data**

In [7]:
"""Clean the data"""
def preprocess(text,stop_words,stemming=False,lemantizing=False):
  
  text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
  #tokenize
  text_list=text.split()
  # text_list=[str(word) for word in text_list]
  #stopwords
  if stop_words:
    text_list=[word for word in text_list if word not in stop_words]
  
  ## Stemming 
  if stemming ==True:
    st=nltk.stem.porter.PorterStemmer()
    text_list=[st.stem(word) for word in text_list]
    
  ## Lemmatisation 
  if lemantizing==True:
    le=nltk.stem.wordnet.WordNetLemmatizer()
    text_list=[le.lemmatize(word) for word in text_list]

   ## tokenize to string again
    text = " ".join(text_list)
    return text
 
    



In [8]:
"""Clean the data"""
stopwords_list=nltk.corpus.stopwords.words("english")
dataset["clean_history"]=dataset["history_text"].apply(lambda x: preprocess(x,stop_words=stopwords_list,stemming=False,lemantizing=True))

labelencoder = LabelEncoder()
dataset["creator_labels"] = labelencoder.fit_transform(dataset["creator"])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


# **Split and load the data**

In [9]:
"""Split the data"""
# X_train,X_test,Y_train,Y_test=train_test_split(self.dataset["clean_history"],self.dataset["creator_labels"],test_size=0.2,random_state=self.RANDOM_SEED)
df_train,df_test=train_test_split(dataset,test_size=0.2,random_state=RANDOM_SEED)
    

In [10]:
"""Encode class to encode the text data into tokens and extract input ids, attention masks that arec necessary for the Bert model"""
class Encode(Dataset):

  def __init__(self,sentences, targets,tokenizer,max_len):
    self.sentences=sentences
    self.targets=targets
    self.tokenizer=tokenizer
    self.max_len=max_len

  def __len__(self):
    return len(self.sentences)

  def __getitem__(self,item):
    Hero_story=self.sentences[item]
    creator_name=self.targets[item]
    encoding=self.tokenizer.encode_plus(Hero_story,max_length=self.max_len,
                                   pad_to_max_length=True,
                                   add_special_tokens=True,
                                   return_token_type_ids=False,
                                   return_tensors='pt',
                                   return_attention_mask=True,
                                   truncation=True)
    
    return{
        'Hero_History': Hero_story,
        'input_ids':encoding['input_ids'].flatten(),
        'attention_mask':encoding['attention_mask'].flatten(),
        'targets':torch.tensor(creator_name,dtype=torch.long)

    }

In [11]:
"""load the data using data loader"""
def data_loader(df):
   
  loader=Encode(sentences=df.clean_history.to_numpy(),targets=df.creator_labels.to_numpy(),tokenizer=tokenizer,max_len=MAX_LEN)
  
  return DataLoader(loader,num_workers=2,batch_size=BATCH_SIZE)

In [12]:
"""load the data using data loader and tune the hyperparameters"""
train_loader=data_loader(df_train)
# test_loader=self.data_loader(df_test)

optimizer=AdamW(model.parameters(),lr=LEARNING_RATE,correct_bias=False)
total_steps=len(train_loader) *EPOCHS
scheduler=get_linear_schedule_with_warmup(optimizer,num_warmup_steps=0,num_training_steps=total_steps)
loss_fn=nn.CrossEntropyLoss().to(device)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


# **Training the model**

In [13]:

def train_epoch(model,
        train_loader,    
        loss_fn, 
        optimizer, 
        scheduler, 
        n_total):    
    
    model = model.train()

    losses = []
    correct_predictions = 0
    
    for d in train_loader:
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )

      _, preds = torch.max(outputs, dim=1)
      loss = loss_fn(outputs, targets)

      correct_predictions += torch.sum(preds == targets)
      losses.append(loss.item())

      loss.backward()
      nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
      optimizer.step()
      scheduler.step()
      optimizer.zero_grad()

    return correct_predictions.double()/n_total, np.mean(losses)

   
  

In [14]:
for epoch in range(EPOCHS):

  print(f"Epoch {epoch +1}/{EPOCHS}")
  print('-'*10)

  train_acc, train_loss = train_epoch(
    model,
    train_loader,    
    loss_fn, 
    optimizer, 
    scheduler, 
    len(df_train)
  )

  print(f"Trainloss {train_loss} accuracy {train_acc}")

Epoch 1/5
----------


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

Trainloss 1.9384416799022728 accuracy 0.40086206896551724
Epoch 2/5
----------


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

Trainloss 1.347890503602485 accuracy 0.6482758620689655
Epoch 3/5
----------


...

Trainloss 0.9936418484335077 accuracy 0.7517241379310344
Epoch 4/5
----------


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

Trainloss 0.8150594046671097 accuracy 0.7862068965517242
Epoch 5/5
----------


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

Trainloss 0.7184853677267897 accuracy 0.8137931034482758


# **Predict**

In [ ]:
def predict(self,model,new_text):


    encoding=self.tokenizer.encode_plus(new_text,max_length=384,
                                    pad_to_max_length=True,
                                    add_special_tokens=True,
                                    return_token_type_ids=False,
                                    return_tensors='pt',
                                    return_attention_mask=True,
                                    truncation=True)
      
      
      
    input_ids=encoding['input_ids'].to(self.device)
    attention_mask=encoding['attention_mask'].to(self.device)
    
    with torch.no_grad():
        # text = torch.tensor(encoding)
        outputs=model(input_ids=input_ids, attention_mask=attention_mask)
        _,preds=torch.max(outputs,dim=1)
        print(preds)

  